Revisions:

* 07/16/2016 -- Updated for ncclient 0.5.2
* 08/20/2016 -- Forked from SEVT demo notebook to do some QoS play

In [ ]:
import logging

handler = logging.StreamHandler()
for l in ['ncclient.transport.ssh', 'ncclient.transport.session', 'ncclient.operations.rpc']:
    logger = logging.getLogger(l)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)

## Connecting to a Device

First, we need to make sure the device itself has NETCONF/YANG enabled. It's pretty simple:

```
ssh server v2
ssh server netconf
netconf-yang agent ssh
```

You also need to generate a host key from the exec prompt:

```
crypto key generate rsa
```

Next, let's define some variables that let us attach to the device. I'm connecting through an SSH tunnel to my device running on a UCS server in the lab. You should replace the variables below with the device of your choice:

In [ ]:
# Running over an SSH tunnel!
HOST = '127.0.0.1'
PORT = 8305
USER = 'cisco'
PASS = 'cisco'

In [ ]:
# XR box on fe-ucs34
HOST = '192.248.175.222'
PORT = 830
USER = 'cisco'
PASS = 'cisco'

Now let's establish a NETCONF session to that box using ncclient. Set the timeout to 600 seconds to allow for long-running operations.

In [ ]:
from ncclient import manager
from lxml import etree

def pretty_print(nc_retval):
    '''The ncclient library works with XML. It takes XML in and gives XML back.
    Yeah, I know it's not JSON, so we need to make it a little easier to read,
    so let's define a pretty printer we can use with ncclient responses
    '''
    print(etree.tostring(etree.fromstring(nc_retval.data_xml), pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True

m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    timeout=600,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

## Schema Discovery

Building on the parsing of the capabilities we saw above, NETCONF/YANG can also let a client discover more details on the schemas supported by a box.

But why do we care? Let's think back to what we talked about earlier. About the need for boxes to describe themselves to their clients. To expose their "model".

Let's pick a base model that looks like it may do something interesting, for example ```Cisco-IOS-XR-ifmgr-cfg```, and let's download the schema. The ncclient library provides a nice, simple function for that.

In [ ]:
from subprocess import Popen, PIPE, STDOUT

SCHEMA_TO_GET = 'Cisco-IOS-XR-qos-ma-oper'
c = m.get_schema(SCHEMA_TO_GET)
p = Popen(['pyang', '-f', 'tree', "--tree-path=qos/interface-table/interface/input", "--tree-depth=7"], stdout=PIPE, stdin=PIPE, stderr=PIPE)
stdout_data = p.communicate(input=c.data)[0]

print stdout_data

## Config Data

In [ ]:
try:
    c = m.get_config(source='running')
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## Oper Data?

As we touched on before, NETCONF also has the ```get``` operation. This can get both configuration state **and** operational state.

### Verbose Interface Stats

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper"/>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Slightly Less Verbose Interface Stats

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper">
  <interface-briefs/>
</interfaces>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Zeroing In On A Specific Interface

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper">
  <interface-xr>
    <interface>
      <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
      <interface-statistics/>
    </interface>
  </interface-xr>
</interfaces>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Zeroing In On Specific Stats Across Interfaces

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-pfi-im-cmd-oper">
  <interface-xr>
    <interface>
      <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
      <interface-statistics>
        <full-interface-stats>
          <packets-received/>
          <bytes-received/>
          <packets-sent/>
          <bytes-sent/>
        </full-interface-stats>
      </interface-statistics>
    </interface>
  </interface-xr>
</interfaces>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### QoS Statistics

In [ ]:
from ncclient.operations import TimeoutExpiredError
from jinja2 import Template
import jxmlease
import json

filter1 = Template('''
<qos xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-qos-ma-oper">
  <interface-table>
    <interface>
      <interface-name>{{INTF_NAME}}</interface-name>
      <input/>
    </interface>
  </interface-table>
</qos>
''')

filter2 = Template('''
<qos xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-qos-ma-oper">
  <interface-table/>
</qos>
''')


filter3 = Template('''
<qos xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-qos-ma-oper">
  <interface-table>
    <interface>
      <input/>
      <output/>
    </interface>
  </interface-table>
</qos>
''')


def display_qos_oper_info(direction, statistics):
    print("  {} Policy Name {}".format(direction, statistics['policy-name']))
    for clazz in statistics['class-stats']:
        print("    Class {}".format(clazz['class-name']))
        print("      General Stats:")
        for k in sorted(clazz['general-stats'].keys()):
            print("        {}".format(k))

try:
    c = m.get(filter=('subtree', filter2.render(INTF_NAME="GigabitEthernet0/0/0/0")))
    root = jxmlease.parse(c.data_xml)
    print(json.dumps(root, indent=1, sort_keys=True))
    for intf in root['data']['qos']['interface-table']['interface']:
        print(intf['interface-name'])
        input_data = intf.get('input')
        if input_data:
            display_qos_oper_info('Input', input_data['statistics'])
        output_data = intf.get('output')
        if output_data:
            display_qos_oper_info('Output', input_data['statistics'])
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)